In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('..')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("poster")
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split, KFold
from merf import *
#from merf.evaluator import plot_bhat, plot_training_stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as seas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from tabulate import tabulate
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import wbgapi as wb
from scipy import stats

seas.set(style="ticks", context="talk")
plt.style.use("dark_background")

In [ ]:
indexes = ['EN.ATM.CO2E.KT',        #CO2 emissions (kt)
           'NY.GDP.MKTP.PP.KD',     #GDP per capita, PPP (constant 2017 international $)
           'SP.POP.TOTL',           #Population, total 
           'EG.FEC.RNEW.ZS',        #Renewable energy consumption (% of total final energy
           'SP.URB.TOTL.IN.ZS',     #Urban population (% of total population) 
           'NV.MNF.TECH.ZS.UN',     #Medium and high-tech manufacturing value added (% manufacturing value added)
           'NE.TRD.GNFS.ZS']        #Trade (% of GDP)

In [ ]:
countries = ['Argentina',
'Armenia',
'Australia',
'Austria',
'Belgium',
'Bulgaria',
'Belarus',
'Brazil',
'Canada',
'Switzerland',
'Chile',
'China',
'Colombia',
'Czech Republic',
'Germany',
'Denmark',
'Spain',
'Estonia',
'Finland',
'France',
'United Kingdom',
'Georgia',
'Greece',
'Croatia',
'Hungary',
'India',
'Ireland',
'Iceland',
'Israel',
'Italy',
'Japan',
'Kazakhstan',
'Korea, Rep.',
'Lithuania',
'Latvia',
'Moldova',
'Mexico',
'Malaysia',
'Netherlands',
'Norway',
'New Zealand',
'Panama',
'Peru',
'Poland',
'Portugal',
'Romania',
'Russian Federation',
'Singapore',
'Serbia',
'Slovak Republic',
'Slovenia',
'Sweden',
'Turkey',
'Ukraine',
'United States',
'South Africa'] 

In [ ]:
countries = [wb.economy.coder(country) for country in countries]

In [ ]:
raw_dataset = wb.data.DataFrame(series = indexes, economy = countries, time = range(1994,2023), labels=True,columns='series')

In [ ]:
raw_dataset.unstack(0).stack()

# Overview

In [ ]:
col = ['Number of variables',
'Number of observations',
'Missing cells',
'Missing cells (%)',
'Rows with Missing cells',
'Rows with Missing cells (%)',
'Duplicate rows',
'Duplicate rows (%)']

val = [len(raw_dataset.columns),
len(raw_dataset),
raw_dataset.isna().values.sum(),
(raw_dataset.isna().values.sum()/len(raw_dataset)) * 100,
len(raw_dataset) - len(raw_dataset.dropna()),
((len(raw_dataset)-len(raw_dataset.dropna()))/len(raw_dataset)) * 100,
raw_dataset.duplicated().sum(),
raw_dataset.duplicated().sum()]

val = list(map(round,val))

overview = pd.DataFrame(zip(col,val), columns=['Variables', 'Count'])

In [ ]:
overview

## Data preprocessing

### Renaming columns for berter readability

In [ ]:
raw_dataset = raw_dataset.rename({'EG.FEC.RNEW.ZS': 'Renewable energy consumption',
'EN.ATM.CO2E.KT': 'CO2 emissions',
'NE.TRD.GNFS.ZS': 'Trade (% of GDP)',
'NV.MNF.TECH.ZS.UN': 'Perc Manufacturing Value Added',
'NY.GDP.MKTP.PP.KD': 'GDP per capita',
'SP.POP.TOTL': 'Total Population',
'SP.URB.TOTL.IN.ZS': 'Urban Population',
},axis = 1)

In [ ]:
raw_dataset.head()

### Missing value check


In [ ]:
raw_dataset.isna().sum()

In [ ]:
raw_dataset = raw_dataset.dropna()

### Outlier detection and treatment

In [ ]:
def outlier_treatment(dataset):
    """
    
    Values lying outside 3SD are considered as outliers for this analysis
    Outlier values are treated with mean of that column
    
    Parameters
    ----------
    df : dataframe
    
    Returns
    ----------
    Dataframe with treated outlier values
    """
    outlier_summary = []

    for col in dataset.columns[2:]:
        outliers = dataset[(np.abs(stats.zscore(dataset[col]))>3)][col]
        outlier_summary.append([col, len(outliers)])
        dataset.loc[outliers.index, col] = dataset[col].mean()

    outlier_summary = pd.DataFrame(outlier_summary, columns=['Variables', 'No. of Outliers'])

    return dataset,outlier_summary

raw_dataset,outlier_summary = outlier_treatment(raw_dataset)

outlier_summary

### Calculating Column of Intrest

In [ ]:
raw_dataset

In [ ]:
raw_dataset['CO2 per Capita'] = raw_dataset['CO2 emissions']/raw_dataset['Total Population']

### Log Trasforming the data

In [ ]:
for col in [col for col in raw_dataset.columns[2:]]:
    raw_dataset[col] = np.log10(raw_dataset[col] + 1)

In [ ]:
processed_df = raw_dataset

## Exploratory Data Analysis

### Data Summary


In [ ]:
processed_df.describe(include='all')

In [ ]:
%matplotlib inline

n_rows=3
n_cols=3
# Create the subplots
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize = (18,15))

colors = {0:'green',1:'red',2:'yellow',3:'blue',4:'orange',5:'pink',6:'brown',7:'gray'}

for i, column in enumerate(processed_df.columns[2:]):
     seas.distplot(processed_df[column],color= colors[i],ax=axes[i//n_cols,i%n_cols])

### Pair Plots

In [ ]:

seas.pairplot(data=processed_df,  hue='Country',palette="Set1")


In [ ]:
seas.pairplot(data=processed_df,  hue='Time',palette="Set1")


## Data Spliting

In [ ]:
X = processed_df.iloc[:,:-1]
y = processed_df.iloc[:,-1]
  
# using the train test split function
X_train, X_test,y_train, y_test = train_test_split(X,y ,
                                   random_state=104, 
                                   test_size=0.20, 
                                   shuffle=True)


## Modeling

### Mixed Effects Random Forest model.  - `y=f(X)+biZ+e`

* y is the target variable. The current code only supports regression for now, e.g. continuously varying scalar value
* X is the fixed effect features. Assume p dimensional
* f(.) is the nonlinear fixed effects mode, e.g. random forest
* Z is the random effect features. Assume q dimensional.
* e is iid noise ~N(0, sigma_e²)
* i is the cluster index. Assume k clusters in the training.
* bi is the random effect coefficients. They are different per cluster i but are assumed to be drawn from the same distribution ~N(0, Sigma_b) where Sigma_b is learned from the data.

### MERF Package

**Parameters**

* X (np.ndarray) – fixed effect covariates
* Z (np.ndarray) – random effect covariates
* clusters (pd.Series) – cluster assignments for samples
* y (np.ndarray) – response/target variable

A lot of data out there has a clustered structure. The most typical example is longitudinal clustering, which occurs when there are many measurements per individual of a phenomenon to be modeled. Assume we wish to model math test scores as a function of sleep parameters, but each student has many measures. The unique student in this situation is a cluster. We employed the MERF model for this project to predict the CO2 per capita values, clustering the 56 countries we have.

In [150]:
max_iter = 200
cv = KFold(n_splits=5, shuffle=True)
mse_mrf_rec = []
mse_mrf_rec_CO2 = []
mse_mrf_rec_CO2_Trade =[]
mse_mrf_rec_CO2_Trade_pmva = []
mse_mrf_rec_CO2_Trade_pmva_GDPpc = []
mse_mrf_rec_CO2_Trade_pmva_GDPpc_tp = []
mse_mrf_rec_CO2_Trade_pmva_GDPpc_tp_up = []

for train_idx, test_idx in cv.split(processed_df):
    # actually split the data
    train = processed_df.iloc[train_idx]
    test = processed_df.iloc[test_idx]
    # ground truth
    y = test['CO2 per Capita']

    ################################################################################
        
    # Mixed Effects Random Forest Training with Renewable energy consumption
    mrf = MERF(max_iterations=max_iter)
    X_train =  train[['Time']]
    Z_train = np.array((np.ones(len(X_train)), train['Renewable energy consumption'])).T
    clusters_train = train['Country']
    y_train = train['CO2 per Capita']
    mrf.fit(X_train, Z_train, clusters_train, y_train)
    
    # Mixed Effects Random Forest Test with Renewable energy consumption
    X_test = test[['Time']]
    Z_test = np.array((np.ones(len(X_test)), test['Renewable energy consumption'])).T
    clusters_test = test['Country']
    yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)
    mse_mrf_rec.append(np.sqrt(np.sum((y - yhat_mrf)**2)) / len(y))

    ################################################################################

    ################################################################################
        
    # Mixed Effects Random Forest Training with Renewable energy consumption & CO2 emissions
    mrf = MERF(max_iterations=max_iter)
    X_train =  train[['Time']]
    Z_train = np.array((np.ones(len(X_train)), train['Renewable energy consumption'],train['CO2 emissions'])).T
    clusters_train = train['Country']
    y_train = train['CO2 per Capita']
    mrf.fit(X_train, Z_train, clusters_train, y_train)
    
    # Mixed Effects Random Forest Test with Renewable energy consumption & CO2 emissions
    X_test = test[['Time']]
    Z_test = np.array((np.ones(len(X_test)), test['Renewable energy consumption'],test['CO2 emissions'])).T
    clusters_test = test['Country']
    yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)
    mse_mrf_rec_CO2.append(np.sqrt(np.sum((y - yhat_mrf)**2)) / len(y))

    ################################################################################

    ################################################################################
        
    # Mixed Effects Random Forest Training with Renewable energy consumption, CO2 emissions & Trade (% of GDP)
    mrf = MERF(max_iterations=max_iter)
    X_train =  train[['Time']]
    Z_train = np.array((np.ones(len(X_train)), train['Renewable energy consumption'],train['CO2 emissions'],train['Trade (% of GDP)'])).T
    clusters_train = train['Country']
    y_train = train['CO2 per Capita']
    mrf.fit(X_train, Z_train, clusters_train, y_train)
    
    # Mixed Effects Random Forest Test with Renewable energy consumption, CO2 emissions & Trade (% of GDP)
    X_test = test[['Time']]
    Z_test = np.array((np.ones(len(X_test)), test['Renewable energy consumption'],test['CO2 emissions'],test['Trade (% of GDP)'])).T
    clusters_test = test['Country']
    yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)
    mse_mrf_rec_CO2_Trade.append(np.sqrt(np.sum((y - yhat_mrf)**2)) / len(y))

    ################################################################################

    ################################################################################
        
    # Mixed Effects Random Forest Training with Renewable energy consumption, CO2 emissions & Trade (% of GDP) & Perc Manufacturing Value Added
    mrf = MERF(max_iterations=max_iter)
    X_train =  train[['Time']]
    Z_train = np.array((np.ones(len(X_train)), train['Renewable energy consumption'],train['CO2 emissions'],train['Trade (% of GDP)'],train['Perc Manufacturing Value Added'])).T
    clusters_train = train['Country']
    y_train = train['CO2 per Capita']
    mrf.fit(X_train, Z_train, clusters_train, y_train)
    
    # Mixed Effects Random Forest Test with Renewable energy consumption, CO2 emissions & Trade (% of GDP) & Perc Manufacturing Value Added
    X_test = test[['Time']]
    Z_test = np.array((np.ones(len(X_test)), test['Renewable energy consumption'],test['CO2 emissions'],test['Trade (% of GDP)'],test['Perc Manufacturing Value Added'])).T
    clusters_test = test['Country']
    yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)
    mse_mrf_rec_CO2_Trade_pmva.append(np.sqrt(np.sum((y - yhat_mrf)**2)) / len(y))

    ################################################################################  

    ################################################################################
        
    # Mixed Effects Random Forest Training with Renewable energy consumption, CO2 emissions & Trade (% of GDP), Perc Manufacturing Value Added & GDP per capita
    mrf = MERF(max_iterations=max_iter)
    X_train =  train[['Time']]
    Z_train = np.array((np.ones(len(X_train)), train['Renewable energy consumption'],train['CO2 emissions'],train['Trade (% of GDP)'],train['Perc Manufacturing Value Added'],train['GDP per capita'])).T
    clusters_train = train['Country']
    y_train = train['CO2 per Capita']
    mrf.fit(X_train, Z_train, clusters_train, y_train)
    
    # Mixed Effects Random Forest Test with Renewable energy consumption, CO2 emissions & Trade (% of GDP), Perc Manufacturing Value Added & GDP per capita
    X_test = test[['Time']]
    Z_test = np.array((np.ones(len(X_test)), test['Renewable energy consumption'],test['CO2 emissions'],test['Trade (% of GDP)'],test['Perc Manufacturing Value Added'],test['GDP per capita'])).T
    clusters_test = test['Country']
    yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)
    mse_mrf_rec_CO2_Trade_pmva_GDPpc.append(np.sqrt(np.sum((y - yhat_mrf)**2)) / len(y))

    ################################################################################ 

    ################################################################################
        
    # Mixed Effects Random Forest Training with Renewable energy consumption, CO2 emissions & Trade (% of GDP), Perc Manufacturing Value Added, GDP per capita & Total Population
    mrf = MERF(max_iterations=max_iter)
    X_train =  train[['Time']]
    Z_train = np.array((np.ones(len(X_train)), train['Renewable energy consumption'],train['CO2 emissions'],train['Trade (% of GDP)'],train['Perc Manufacturing Value Added'],train['GDP per capita'],train['Total Population'])).T
    clusters_train = train['Country']
    y_train = train['CO2 per Capita']
    mrf.fit(X_train, Z_train, clusters_train, y_train)
    
    # Mixed Effects Random Forest Test with Renewable energy consumption, CO2 emissions & Trade (% of GDP), Perc Manufacturing Value Added, GDP per capita & Total Population
    X_test = test[['Time']]
    Z_test = np.array((np.ones(len(X_test)), test['Renewable energy consumption'],test['CO2 emissions'],test['Trade (% of GDP)'],test['Perc Manufacturing Value Added'],test['GDP per capita'],test['Total Population'])).T
    clusters_test = test['Country']
    yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)
    mse_mrf_rec_CO2_Trade_pmva_GDPpc_tp.append(np.sqrt(np.sum((y - yhat_mrf)**2)) / len(y))

    ################################################################################ 

    ################################################################################
        
    # Mixed Effects Random Forest Training with Renewable energy consumption, CO2 emissions & Trade (% of GDP), Perc Manufacturing Value Added, GDP per capita, Total Population & Urban Population
    mrf = MERF(max_iterations=max_iter)
    X_train =  train[['Time']]
    Z_train = np.array((np.ones(len(X_train)), train['Renewable energy consumption'],train['CO2 emissions'],train['Trade (% of GDP)'],train['Perc Manufacturing Value Added'],train['GDP per capita'],train['Total Population'],train['Urban Population'])).T
    clusters_train = train['Country']
    y_train = train['CO2 per Capita']
    mrf.fit(X_train, Z_train, clusters_train, y_train)
    
    # Mixed Effects Random Forest Test with Renewable energy consumption, CO2 emissions & Trade (% of GDP), Perc Manufacturing Value Added, GDP per capita, Total Population & Urban Population
    X_test = test[['Time']]
    Z_test = np.array((np.ones(len(X_test)), test['Renewable energy consumption'],test['CO2 emissions'],test['Trade (% of GDP)'],test['Perc Manufacturing Value Added'],test['GDP per capita'],test['Total Population'],test['Urban Population'])).T
    clusters_test = test['Country']
    yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)
    mse_mrf_rec_CO2_Trade_pmva_GDPpc_tp_up.append(np.sqrt(np.sum((y - yhat_mrf)**2)) / len(y))

    ################################################################################ 

INFO     [merf.py:307] Training GLL is -3549.8141701751188 at iteration 1.
INFO     [merf.py:307] Training GLL is -6773.130765591915 at iteration 2.
INFO     [merf.py:307] Training GLL is -9877.602393062227 at iteration 3.
INFO     [merf.py:307] Training GLL is -12950.300067287644 at iteration 4.
INFO     [merf.py:307] Training GLL is -15967.146376315219 at iteration 5.
INFO     [merf.py:307] Training GLL is -18537.34938299054 at iteration 6.
INFO     [merf.py:307] Training GLL is -19539.875431822296 at iteration 7.
INFO     [merf.py:307] Training GLL is -19625.9669886366 at iteration 8.
INFO     [merf.py:307] Training GLL is -19632.79397683387 at iteration 9.
INFO     [merf.py:307] Training GLL is -19634.171725628457 at iteration 10.
INFO     [merf.py:307] Training GLL is -19634.579762604266 at iteration 11.
INFO     [merf.py:307] Training GLL is -19634.81597395814 at iteration 12.
INFO     [merf.py:307] Training GLL is -19634.909292876113 at iteration 13.
INFO     [merf.py:307] Train